In [1]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# URL que vamos dar scrap, no caso do projeto, peguei um link de um produto da Amazon
url = 'https://www.amazon.com.br/Elden-Ring-Padr%C3%A3o-Playstation-4/dp/B07T8XCPSP/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=VM464FQOFRUJ&keywords=elden+ring&qid=1684120907&sprefix=elden+ring%2Caps%2C221&sr=8-1&ufe=app_do%3Aamzn1.fos.6a09f7ec-d911-4889-ad70-de8dd83c8a74'


# http://www.networkinghowtos.com/howto/common-user-agent-list/
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0',
            'Accept-Language': 'en-US, en;q=0.5'})


# Vamos mandar um 'GET request' para a URL, para nos pegarmos o HTML
response = requests.get(url, headers= HEADERS)
html_content = response.text

# BeautifulSoup object, ele que vai receber o html que pegamos
soup = BeautifulSoup(html_content, 'html.parser')

# Extrair o titulo
product_title = soup.find('span', attrs={'id': 'productTitle'}).get_text().strip()

# Preço
price = soup.find("span", attrs={'class': 'a-offscreen'}).string.strip()

# Score (Nota, ou rating, como vocês preferirem)
rating = soup.find('span', attrs={'class': 'a-icon-alt'}).get_text().strip()

# Reviews (Nesse caso é quantidade)
reviews_count = soup.find('span', attrs={'id': 'acrCustomerReviewText'}).get_text().strip()

# Descrição do produto
description_element = soup.find('div', attrs={'id': 'productDescription'})
description = description_element.get_text().strip() if description_element else 'N/A'

# Printzao qualidade
print('Product Title: {}'.format(product_title))
print('Price: {}'.format(price))
print('Rating: {}'.format(rating))
print('Reviews Count: {}'.format(reviews_count))
print('Description: {}'.format(description))


Product Title: Elden Ring - PlayStation 4
Price: R$219,99
Rating: 4,9 de 5 estrelas
Reviews Count: 3.956 avaliações de clientes
Description: A Ordem Áurea foi quebrada. Levante-se, Maculado, e seja guiado pela graça para empunhar o poder do Anel Prístino e se tornar um Lorde Prístino nas Terras Intermédias. ELDEN RING, é um RPG de ação e fantasia ambientado em um mundo cheio de mistério e perigo. Um novo mundo de fantasia: Viaje pelas Terras Intermédias, um mundo de fantasia criado por Hidetaka Miyazaki, criador de DARK SOULS, e George R.R. Martin, autor do best-seller, AS CRÔNICAS de GELO e FOGO.Desvende os mistérios do poder do Anel Prístino. Encontre adversários com histórias profundas, personagens com suas próprias motivações únicas para ajudar ou atrapalhar seu progresso, e criaturas terríveis.


In [15]:
# Vou criar uma variavel com data, para salvar no nosso dataset quando pegamos esses dados

import datetime
now = datetime.datetime.now()
print(now)
print(now.year, now.month, now.day, now.hour, now.minute)

2023-05-15 16:55:08.969528
2023 5 15 16 55


In [5]:
# Temos muitas variaves...vou anotar tudo aqui so pra eu não esquecer
# product_title, price, rating, reviews_count, description, now ---> Nessa ordem
import pandas as pd

d = {'Titulo': product_title, 'Preço': price, 'Rating': rating, 'Reviews': reviews_count, 'Descrição': description, 'Data_request': now}
df = pd.DataFrame(data = d, index = [0])


In [6]:
display(df.head())

,Titulo,Preço,Rating,Reviews,Descrição,Data_request
0,Elden Ring - PlayStation 4,"R$219,99","4,9 de 5 estrelas",3.956 avaliações de clientes,"A Ordem Áurea foi quebrada. Levante-se, Macula...",2023-05-15


In [7]:
# Ja temos nossa dataset...agora vamos criar uma função para brincar com as urls!
# Ou seja, vou criar uma função para no momento que jogarmos a url nela, ela ja vai salvar no dataset e depois printar

def get_amazon(x):
        
    url = x
    # http://www.networkinghowtos.com/howto/common-user-agent-list/
    HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})


    # Vamos mandar um 'GET request' para a URL, para nos pegarmos o HTML
    response = requests.get(url, headers= HEADERS)
    html_content = response.text

    # BeautifulSoup object, ele que vai receber o html que pegamos
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extrair o titulo
    product_title = soup.find('span', attrs={'id': 'productTitle'}).get_text().strip()

    # Preço
    price = soup.find("span", attrs={'class': 'a-offscreen'}).string.strip()

    # Score (Nota, ou rating, como vocês preferirem)
    rating = soup.find('span', attrs={'class': 'a-icon-alt'}).get_text().strip()

    # Reviews (Nesse caso é quantidade)
    reviews_count = soup.find('span', attrs={'id': 'acrCustomerReviewText'}).get_text().strip()

    # Descrição do produto
    description_element = soup.find('div', attrs={'id': 'productDescription'})
    description = description_element.get_text().strip() if description_element else 'N/A'

    # Printzao qualidade
    print('Product Title: {}'.format(product_title))
    print('Price: {}'.format(price))
    print('Rating: {}'.format(rating))
    print('Reviews Count: {}'.format(reviews_count))
    print('Description: {}'.format(description))
    
    # Colocando a data tb, ninguem é de ferro
    now = datetime.date.today()
    
    # Salvando no nosso dataframe do pandinhas
    new_row = {'Titulo': product_title, 'Preço': price, 'Rating': rating, 'Reviews': reviews_count, 'Descrição': description, 'Data_request': now}
    global df
    df = df.append(new_row, ignore_index=True)

In [8]:
get_amazon('https://www.amazon.com.br/Boa-Noite-Punpun-Inio-Asano/dp/8545709617/ref=pd_ci_mcx_mh_mcx_views_0?pd_rd_w=Uo8xi&content-id=amzn1.sym.5750b988-c722-40d9-8c9a-41af4853947c&pf_rd_p=5750b988-c722-40d9-8c9a-41af4853947c&pf_rd_r=B2BT9G83ZXDFPS7CCS8Y&pd_rd_wg=2qdbD&pd_rd_r=9b803702-b785-42aa-adec-6f919c7bc450&pd_rd_i=8545709617')

Product Title: Boa Noite Punpun - Vol. 1
Price: R$ 49,90
Rating: 4,8 de 5 estrelas
Reviews Count: 2.727 avaliações de clientes
Description: N/A


In [9]:
df.head()

,Titulo,Preço,Rating,Reviews,Descrição,Data_request
0,Elden Ring - PlayStation 4,"R$219,99","4,9 de 5 estrelas",3.956 avaliações de clientes,"A Ordem Áurea foi quebrada. Levante-se, Macula...",2023-05-15
1,Boa Noite Punpun - Vol. 1,"R$ 49,90","4,8 de 5 estrelas",2.727 avaliações de clientes,N/A,2023-05-15


In [10]:
# Depois de brincar com os links, é so salvar
# Esse primeiro codigo foi a primeira vez que fiz, para criar o csv -> Amazon_scrap
# Ele ja existe, agora é so 'adicionar' os novos dados no csv que ja criei

# df.to_csv('Amazon_scrap.csv', sep=',', index=False, encoding='utf-8')



# Agora vem o adicionar no csv ja existente
df.to_csv('Amazon_scrap.csv', mode='a', index=False, header=False)



# Segundo projeto

### No segundo projeto eu criei um arquivo (price_check.py)
##### Ele faz a mesma coisa que estamos fazendo agora, a diferença que mandei ele verificar os preços de produtos fixos
##### Esses produtos estão com as urls salvas no csv (urls) e toda vez que o programa é executado ele verifica o preço dos mesmos produtos
##### O objetivo era fazer com que ele rodasse algumas vezes no dia, por alguns dias, para verificar a variação no preço
##### Mas acabou que não executei varias vezes, pois os produtos que escolhi não sofrem tanta variação assim...
##### Talvez eu tente fazer novamente, acessando outro site com algum produto que tenha um valor variavel
